# Load gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_dir="/content/drive/MyDrive/Semester-Docs/Sem5/Capstone/"
proj_dir=root_dir+"chest_xray2/"
nor_dir=proj_dir+"train/NORMAL"
pneu_dir=proj_dir+"train/PNEUMONIA"

In [ ]:
import pickle
with open(proj_dir+'chest_xray_pneumonia.pkl_final', 'rb') as file:
  merged_df = pickle.load(file)

In [ ]:
print(merged_df)

               file_name                                          file_path  \
0         NORMAL_453.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
1         NORMAL_404.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
2         NORMAL_278.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
3         NORMAL_410.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
4         NORMAL_513.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
...                  ...                                                ...   
3597  PNEUMONIA_1423.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
3598  PNEUMONIA_1465.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
3599  PNEUMONIA_1651.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
3600  PNEUMONIA_1579.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
3601   PNEUMONIA_141.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   

      label                                        

In [ ]:
import os
import numpy as np
import pandas as pd
def create_file_map(root_dir, limit=100):
    file_map = {}  # Initialize an empty map

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            full_path = os.path.join(foldername, filename)
            file_map[filename] = full_path
            if len(file_map) == limit:
              return file_map
    return file_map

nor_file_map = create_file_map(nor_dir, 100000)
pneu_files_map = create_file_map(pneu_dir, 100000)

nor_df = pd.DataFrame(nor_file_map.items(), columns=['file_name', 'file_path'])
nor_label = pd.DataFrame({'label': np.zeros(len(nor_df))})
nor_df_new = pd.concat([nor_df, nor_label], axis=1)

pneu_df = pd.DataFrame(pneu_files_map.items(), columns=['file_name', 'file_path'])
pneu_label = pd.DataFrame({'label': np.ones(len(pneu_df))})
pneu_df_new = pd.concat([pneu_df, pneu_label], axis=1)

merged_df = pd.concat([nor_df_new, pneu_df_new], ignore_index=True)

print(merged_df)

               file_name                                          file_path  \
0         NORMAL_453.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
1         NORMAL_404.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
2         NORMAL_278.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
3         NORMAL_410.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
4         NORMAL_513.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
...                  ...                                                ...   
3597  PNEUMONIA_1423.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
3598  PNEUMONIA_1465.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
3599  PNEUMONIA_1651.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
3600  PNEUMONIA_1579.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   
3601   PNEUMONIA_141.png  /content/drive/MyDrive/Semester-Docs/Sem5/Caps...   

      label  
0       0.0  
1       0.0  
2       0

# Download all tools

In [ ]:
!git clone https://github.com/subrat-mishra/MedSam-Utils
import sys
sys.path.append(root_dir)
sys.path.append("MedSam-Utils")

fatal: destination path 'MedSam-Utils' already exists and is not an empty directory.


In [ ]:
!pip3 install tensorboardX
!pip3 install git+https://github.com/bowang-lab/MedSAM.git
!pip install umap-learn
!pip install pydicom

  Cloning https://github.com/bowang-lab/MedSAM.git to /tmp/pip-req-build-2me6ozul
  Running command git clone --filter=blob:none --quiet https://github.com/bowang-lab/MedSAM.git /tmp/pip-req-build-2me6ozul
  Resolved https://github.com/bowang-lab/MedSAM.git to commit 861d42440061c704762a9bb9a574e7c31d8d751a
  Preparing metadata (setup.py) ... done


In [ ]:
# download model and data
img_id = '1Qf0IQF1zTwOJ4Fh4Ix5rtVhR9cnNbI2M'
!gdown $img_id
model_id = "1UAmWL88roYR7wKlnApw5Bcuzf2iQgk6_&confirm=t"
!gdown $model_id

Downloading...
From: https://drive.google.com/uc?id=1Qf0IQF1zTwOJ4Fh4Ix5rtVhR9cnNbI2M
To: /content/img_demo.png
100% 87.9k/87.9k [00:00<00:00, 85.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UAmWL88roYR7wKlnApw5Bcuzf2iQgk6_
To: /content/medsam_vit_b.pth
100% 375M/375M [00:02<00:00, 126MB/s]


# MedSam Embeddings

In [ ]:
import csv
import os
import pickle
import numpy as np
import pandas as pd
import pydicom
import time
import torch
import torch.nn as nn
from segment_anything import sam_model_registry
from skimage import io, transform
from PIL import Image
import torch.nn.functional as F

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MedSAM_CKPT_PATH = "medsam_vit_b.pth"
medsam_model = sam_model_registry['vit_b'](checkpoint=MedSAM_CKPT_PATH)
medsam_model = medsam_model.to(device)
medsam_model.eval()

def get_medsam_embeddings(img_3c):
  img_1024 = transform.resize(img_3c, (1024, 1024), order=3, preserve_range=True, anti_aliasing=True).astype(np.uint8)
  img_1024 = (img_1024 - img_1024.min()) / np.clip(
      img_1024.max() - img_1024.min(), a_min=1e-8, a_max=None
  )
  # convert the shape to (3, H, W)
  img_1024_tensor = torch.tensor(img_1024).float().permute(2, 0, 1).unsqueeze(0).to(device)

  with torch.no_grad():
      image_embedding = medsam_model.image_encoder(img_1024_tensor) # (1, 256, 64, 64)
  return image_embedding

def get_knee_mri_embedding(file_path):
  metadata_csv_path = root_dir + "KneeMRI/metadata.csv"
  metadata = np.genfromtxt(metadata_csv_path, delimiter=',', names=True,
        dtype='i4,i4,i4,i4,i4,i4,i4,i4,i4,i4,U20')
  file_name = os.path.basename(file_path)
  exam = metadata[metadata['volumeFilename'] == file_name]
  with open(file_path, 'rb') as file_handler: # Must use 'rb' as the data is binary
      volumetric_data = pickle.load(file_handler)
  z_start = exam['roiZ']
  depth = exam['roiDepth']
  mid_depth = depth if depth == 1 else (int)(depth/2)
  img_np = volumetric_data[z_start + mid_depth, :, :]
  img_np = img_np.reshape(-1, img_np.shape[-1])
  return img_np

def get_image_arr(file_path):
  if file_path.endswith("npy"):
    np_load = np.load(file_path)
    mid = (int)(np_load.shape[0]/2) - 1
    img_np =  np_load[mid] # Mid slice
  elif file_path.endswith("pck"):
    # Special case for KneeMRI image
    img_np = get_knee_mri_embedding(file_path)
  elif file_path.endswith("dcm"):
    img_np = pydicom.dcmread(file_path).pixel_array
  else:
    image = Image.open(file_path)
    img_np = np.array(image)

  return img_np

def get_output_tensor(file_path):
  img_np = get_image_arr(file_path)
  if len(img_np.shape) == 2:
      img_3c = np.repeat(img_np[:, :, None], 3, axis=-1)
  else:
      img_3c = img_np
  embeddings = get_medsam_embeddings(img_3c)
  global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
  output_tensor = global_avg_pool(embeddings)
  output_tensor = output_tensor.view(embeddings.size(1))
  return output_tensor.cpu().numpy()

def compute_embedding(index, total, file_path, start_time):
  output_tensor_arr = get_output_tensor(file_path)
  if index % 10 == 0:
    print(f"Time taken: {index}/{total} is {time.time() - start_time:.4f} seconds")
  return output_tensor_arr

In [ ]:
start_time = time.time()

merged_df_copy = merged_df.copy()
batch_size = 100
file_batch_size = batch_size
start = 0
# end = len(merged_df_copy)
end = 1600
for i in range(start, end, batch_size):
  merged_df_copy.loc[i:i+batch_size, 'embeddings'] = merged_df_copy.loc[i:i+batch_size].apply(lambda row: compute_embedding(row.name, len(merged_df), row['file_path'], start_time), axis=1)

  if (i + batch_size) % file_batch_size == 0:
    pkl_file_name = 'chest_xray_pneumonia.pkl_'+str(i + batch_size)
    print("Writing file=", pkl_file_name)
    merged_df_copy.to_pickle(proj_dir+pkl_file_name)

pkl_file_name = 'chest_xray_pneumonia.pkl_final'
print("Writing file=", pkl_file_name)
merged_df_copy.to_pickle(proj_dir+pkl_file_name)

merged_df = merged_df_copy

Time taken: 0/3602 is 8.5654 seconds
Time taken: 10/3602 is 38.2722 seconds
Time taken: 20/3602 is 67.9714 seconds
Time taken: 30/3602 is 97.6388 seconds
Time taken: 40/3602 is 127.0343 seconds
Time taken: 50/3602 is 155.5830 seconds
Time taken: 60/3602 is 183.9461 seconds
Time taken: 70/3602 is 212.7280 seconds
Time taken: 80/3602 is 241.4408 seconds
Time taken: 90/3602 is 270.8805 seconds
Time taken: 100/3602 is 299.8329 seconds
Writing file= chest_xray_pneumonia.pkl_100
Time taken: 100/3602 is 302.6759 seconds
Time taken: 110/3602 is 331.3180 seconds
Time taken: 120/3602 is 359.6343 seconds
Time taken: 130/3602 is 388.0832 seconds
Time taken: 140/3602 is 417.0036 seconds
Time taken: 150/3602 is 445.9835 seconds
Time taken: 160/3602 is 474.4714 seconds
Time taken: 170/3602 is 501.6607 seconds
Time taken: 180/3602 is 531.2456 seconds
Time taken: 190/3602 is 559.3312 seconds
Time taken: 200/3602 is 588.5325 seconds
Writing file= chest_xray_pneumonia.pkl_200
Time taken: 200/3602 is 590.

# For all exams

In [ ]:
# import pickle
# with open('data.pkl', 'rb') as file:
#     merged_df = pickle.load(file)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pandas as pd

file_names = merged_df['file_name']
embeddings = merged_df['embeddings']
labels = merged_df['label'].to_numpy()
labels = ['Tear' if elem > 0 else 'No Tear' for elem in labels]

embeddings = np.stack(embeddings)

tsne = TSNE(n_components=2)

# Fit the t-SNE model to the embedding feature
tsne_results = tsne.fit_transform(embeddings)

df = pd.DataFrame({'x_axis': tsne_results[:, 0], 'y_axis': tsne_results[:, 1],
                   'labels':labels, 'file_names': file_names},
                  columns=['x_axis', 'y_axis', 'labels', 'file_names'])
fig = px.scatter(df, x="x_axis", y="y_axis", color="labels", hover_name="file_names")
fig.show()

In [ ]:
merged_df.to_pickle('chest_xray_pneumonia_train.pkl')

# Debug Images

In [ ]:
# import pickle
# import os
# import re
# import time

# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.patches as patch

# file_map = create_file_map(proj_dir)
# view_images = ["496580-5.pck_1", "425707-8.pck_1"]

# # path to metadata csv file
# metadata_csv_path = proj_dir+'metadata.csv'
# metadata_csv_path = proj_dir+'metadata_vol0.csv'
# metadata = np.genfromtxt(metadata_csv_path, delimiter=',', names=True,
#     dtype='i4,i4,i4,i4,i4,i4,i4,i4,i4,i4,U20,U20,U20,U20,U20,U20,U20')

# print('Column names:')
# print(metadata.dtype.names)

# pattern = r'\d+'
# for each_image in view_images:
#   parts = re.findall(pattern, each_image)
#   curr_examId = int(parts[0])
#   curr_depth = int(parts[2])
#   exams = metadata[metadata['examId'] == curr_examId]
#   exam = exams[0]

#   vol_data_file = exam['volumeFilename']

#   vol_data_path = file_map.get(vol_data_file)
#   with open(vol_data_path, 'rb') as file_handler: # Must use 'rb' as the data is binary
#       volumetric_data = pickle.load(file_handler)

#   # print('\nShape of volume "%s":' % vol_data_path, volumetric_data.shape)
#   print('\n Exam %s:' % (exam))

#   # Get all roi slices from volume
#   z_start = exam['roiZ']
#   depth = exam['roiDepth'] + curr_depth

#   for z in range(z_start, z_start + depth):

#     slice = volumetric_data[z, :, :]

#     # Get roi dimensions
#     x, y, w, h = [exam[attr] for attr in ['roiX', 'roiY', 'roiWidth', 'roiHeight']]

#     # Extract ROI
#     roi = slice[y:y+h, x:x+w]

#     # Plot slice and roi
#     figure = plt.figure()
#     plot = plt.subplot2grid((1, 4), (0, 0), 1, 3) # This makes the slice plot larger than roi plot
#     plot.add_patch(patch.Rectangle((x, y), w, h, fill=None, color='red'))
#     plot.imshow(slice, cmap='gray')
#     plot = plt.subplot2grid((1, 4), (0, 3), 1, 1)
#     plot.imshow(roi, cmap='gray')

#     plt.show()
#     break